# Configuration basics

ablator trainer requires wrapper and config (RunConfig) to launch experiment.

- Training can either be done with a single process or ablation experiment (multiple, or distributed, processes)

- Configuration is divided into different categories: model configs, running configs (either ordinary running configs or parallel training configs)

Ablator has the ability to dynamically create a hierarchical configuration by composition and override it through config files and the command line.

Important configuration classes are presented below. Note that you can either store all these parameters in a yaml config file, or you can play around with classes. You can refer to (these examples)[] to see how these 2 cases are done in practice.

## RunConfig
This configuration class is passed as an argument when initializing the trainer object

| Parameter           | Usage                                                                                                                                                                                                                                                                                                             |
|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| experiment_dir      | location to store experiment artifacts.                                                                                                                                                                                                                                                                           |
| random_seed         | random seed.                                                                                                                                                                                                                                                                                                      |
| train_config        | training configuration. (check ``TrainConfig`` for more details)                                                                                                                                                                                                                                                  |
| model_config        | model configuration. (check ``ModelConfig`` for more details)                                                                                                                                                                                                                                                     |
| keep_n_checkpoints  | number of latest checkpoints to keep.                                                                                                                                                                                                                                                                             |
| tensorboard         | whether to use tensorboardLogger.                                                                                                                                                                                                                                                                                 |
| amp                 | whether to use automatic mixed precision when running on gpu.                                                                                                                                                                                                                                                     |
| device              | device to run on.                                                                                                                                                                                                                                                                                                 |
| verbose             | verbosity level.                                                                                                                                                                                                                                                                                                  |
| eval_subsample      | fraction of the dataset to use for evaluation.                                                                                                                                                                                                                                                                    |
| metrics_n_batches   | max number of batches stored in every tag(train, eval, test) for evaluation.                                                                                                                                                                                                                                      |
| metrics_mb_limit    | max number of megabytes stored in every tag(train, eval, test) for evaluation.                                                                                                                                                                                                                                    |
| early_stopping_iter | The maximum allowed difference between the current iteration and the last iteration with the best metric before applying early stopping. Early stopping will be triggered if the difference ``(current_itr - best_itr)`` exceeds ``early_stopping_iter``. If set to ``None``, early stopping will not be applied. |
| eval_epoch          | The epoch interval between two evaluations.                                                                                                                                                                                                                                                                       |
| log_epoch           | The epoch interval between two logging.                                                                                                                                                                                                                                                                           |
| init_chkpt          | path to a checkpoint to initialize the model with.                                                                                                                                                                                                                                                                |
| warm_up_epochs      | number of epochs marked as warm up epochs.                                                                                                                                                                                                                                                                        |
| divergence_factor   | if ``cur_loss > best_loss > divergence_factor``, the model is considered to have diverged.                                                                                                                                                                                                                        |

## ParallelConfig


## TrainConfig


## ModelConfig


## OptimizerConfig


## SchedulerConfig
